In [40]:
## IMPORTS
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import date, timedelta
from yahoo_fin.stock_info import get_data

In [41]:
## Constant
CLOSE = 0
VOLUME = 1
TICKER = 2
FIVE_YEAR_HIGH = 3
WMA30 = 4
WMA30_SLOPE = 5
VOLUME_PERC = 6
PERCENT = 7
RS = 8
PEAK = 9
TROUGH = 10
SUPPORT = 11
INITIAL_SUPPORT = 12
STAGE = 13
SECOND_BUY = 14
RESISTANCE = 15
SHORT_PEAK = 16
SHORT_TROUGH = 17

In [42]:
spdf = pd.read_pickle("stockData/Spy.pkl")
sectorOfTicker = pd.read_pickle("stockData/nasdaq.pkl")
sectorOfNyse = pd.read_pickle("stockData/nyse.pkl")
sectorOfTicker.update(sectorOfNyse)

## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [43]:
def checkIfStage4(
    i,price,volumePerc, RS, slope, wMA30,prevStage,prevClose,prevResistance,prevPeak,prevTrough,dfSorted,param):
    if slope == 0:
        return "No Enough Data"
    if prevStage == "Stage 4" or prevStage == "Sell Short":
        if price < prevPeak:
            # update shortPeak and shortTrough
            dfSorted.iat[i,SHORT_PEAK] = price
            dfSorted.iat[i,SHORT_TROUGH] = price
        else:
            dfSorted.iat[i,SHORT_PEAK] = prevPeak
            dfSorted.iat[i,SHORT_TROUGH] = min(prevTrough, price)
        if price >= prevResistance*param[5] or price > wMA30*param[6]:
            return "Cover Buy"

        dfSorted.iat[i,RESISTANCE] = prevResistance

        if price == dfSorted.iat[i,SHORT_TROUGH] and prevResistance < prevTrough*param[6]:
            dfSorted.iat[i,RESISTANCE] = prevPeak
        return "Stage 4"
        
    if price > wMA30*param[0]:
        return "Price"
    if slope > param[1]:
        return "Slope"
    if volumePerc < param[2]:
        return "volume"
    if RS > param[3]:
        return "RS"
    if price < prevClose:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price
    else:
        dfSorted.iat[i,SHORT_PEAK] = price
        dfSorted.iat[i,SHORT_TROUGH] = price*0.95
    
    return "Sell Short"
  

In [44]:
def returnStageDfShort(dfSorted,param):
    fullPrint(dfSorted)
    first = True
    for index in dfSorted.index:
        if first:
            if dfSorted.index.get_loc(index) == 0:
                first = False
                continue
        prevIndex = index - timedelta(weeks=1)
        i = dfSorted.index.get_loc(index)
        dfSorted.iat[i, STAGE] = checkIfStage4(i,dfSorted.iat[i,CLOSE],dfSorted.iat[i,VOLUME_PERC],dfSorted.iat[i,RS],dfSorted.iat[i,WMA30_SLOPE],dfSorted.iat[i,WMA30],dfSorted.iat[i-1,STAGE],dfSorted.iat[i-1,CLOSE],dfSorted.iat[i-1,SUPPORT],dfSorted.iat[i,PEAK],dfSorted.iat[i-1,TROUGH],dfSorted,param)
    first = True
    #to_csv
    dfSorted.to_csv("testShortDf.csv")
    return dfSorted[["close","Stage"]]

def getStageShort(ticker,param):
#     today = date.today()
#     # #200->1000
    
    # today = today.strftime('%Y-%m-%d')
    # startDate = startDate.strftime('%Y-%m-%d')
    # df = get_data(ticker, start_date=startDate, end_date=today, index_as_date = True, interval="1wk")
    df = pd.read_pickle("stockData/nyseNasdaq/"+ticker+".pkl")
    return returnStageDfShort(df,param)

In [45]:
testDf = getStageShort("MRNA",[1.05, 1, 0.1,-0.2,1.025,1.05,1.1])

                 close     volume ticker  fiveYearHigh       WMA30  \
2018-12-03   22.000000   12995600   MRNA           0.0         NaN   
2018-12-10   22.000000   12614200   MRNA           0.0         NaN   
2018-12-17   22.000000    8572100   MRNA           0.0         NaN   
2018-12-24   22.000000    5181800   MRNA           0.0         NaN   
2018-12-31   22.000000    6553500   MRNA           0.0         NaN   
2019-01-07   22.000000    5733400   MRNA           0.0         NaN   
2019-01-14   22.000000    2299500   MRNA           0.0         NaN   
2019-01-21   22.000000    2684600   MRNA           0.0         NaN   
2019-01-28   22.000000    4884600   MRNA           0.0         NaN   
2019-02-04   22.000000    3179700   MRNA           0.0         NaN   
2019-02-11   22.000000    5863300   MRNA           0.0         NaN   
2019-02-18   22.000000    3754800   MRNA           0.0         NaN   
2019-02-25   22.000000    7371700   MRNA           0.0         NaN   
2019-03-04   22.4699